# Enrichissement des données géographiques

In [1]:
import sys
import numpy as np
import pandas as pd
import openrouteservice as ors

## Import du dataset

In [2]:
data = pd.read_pickle('store/after_analyze.pkl.xz')
data.reset_index(inplace=True, drop=True)
data.head()

,date_mutation,valeur_fonciere,adresse_numero,adresse_suffixe,adresse_nom_voie,adresse_code_voie,code_postal,nom_commune,lot1_surface_carrez,type_local,surface_reelle_bati,nombre_pieces_principales,surface_terrain,longitude,latitude
0,2014-01-02,194400.0,23,NaN,BD EMILE ROMANET,2794,44100,Nantes,NaN,Appartement,84.0,4,NaN,-1.603261,47.209692
1,2014-01-02,107000.0,11,NaN,RUE DU DOCTEUR ALFRED CORLAY,0882,44800,Saint-Herblain,45.80,Appartement,46.0,2,NaN,-1.644254,47.207462
2,2014-01-09,208154.0,38,NaN,RUE DE LA PLANCHE AU GUE,5555,44300,Nantes,103.64,Appartement,103.0,5,NaN,-1.519753,47.272364
3,2014-01-06,79000.0,8,NaN,RUE DES CARMELITES,1436,44000,Nantes,26.45,Appartement,25.0,1,NaN,-1.551322,47.216626
4,2014-01-02,335000.0,23,NaN,RUE DES CANARIS,1390,44300,Nantes,NaN,Maison,118.0,5,562.0,-1.506824,47.231122


## Recherche des points d'interêt

In [59]:
ISOCHRONE_CLIENT = ors.Client(base_url='http://192.168.1.250:9090/ors')
POIS_CLIENT = ors.Client(base_url='http://192.168.1.250:5000')

ISOCHRONE_PROFILE = 'foot-walking'
ISOCHRONE_NB_MINS = 5

TEST_LOCATION = [[-1.603261,47.209692]]

In [4]:
def get_isochrone(client, location, profile='driving-car',nb_mins=5):
    params = {
        'profile':profile,
        'locations':location,
        'range':[nb_mins * 60],
        'range_type':'time'
    }
    return client.isochrones(**params)

In [5]:
def get_pois(client, geometry, request='pois'):
    params = {
        'request':request,
        'geojson':geometry
    }
    return client.places(**params)

In [81]:
def add_pois_stats(data, profile, nb_mins, isochrone_client=ISOCHRONE_CLIENT, pois_client=POIS_CLIENT):
    results = pd.DataFrame(dtype=np.int8)
    print(data.shape)
    for i, row in data.iterrows():
        location = [[row.longitude, row.latitude]]
        isochrome = get_isochrone(isochrone_client, location, profile, nb_mins)
        geometry = isochrome['features'][0]['geometry']
        stat_pois = get_pois(pois_client, geometry, 'stats')['places']
        stat_pois.pop('total_count', None)
        tmp = dict()
        for cat, children in stat_pois.items():
            for scat, info in children['categories'].items():
                tmp['{}/{}'.format(cat,scat)] = info['count']
        pois_place = pd.concat([row, pd.Series(tmp)])
        pois_place = pois_place.to_frame().T
        results = pd.concat([results, pois_place], sort=False, ignore_index=True)
        
        sys.stdout.write('\r')
        sys.stdout.write('{}%' .format(round(100*(i+1)/data.shape[0],2)))
        sys.stdout.flush()
        
    return results


def add_pois_stats(data, profile, nb_mins, isochrone_client=ISOCHRONE_CLIENT, pois_client=POIS_CLIENT):
    results = pd.DataFrame(dtype=np.int8)
    print(data.shape)
    for i, row in data.iterrows():
        location = [[row.longitude, row.latitude]]
        isochrome = get_isochrone(isochrone_client, location, profile, nb_mins)
        geometry = isochrome['features'][0]['geometry']
        stat_pois = get_pois(pois_client, geometry, 'stats')['places']
        stat_pois.pop('total_count', None)
        tmp = dict()
        for cat, children in stat_pois.items():
            for scat, info in children['categories'].items():
                tmp['{}/{}'.format(cat,scat)] = info['count']
        pois_place = pd.concat([row, pd.Series(tmp)])
        pois_place = pois_place.to_frame().T
        results = pd.concat([results, pois_place], sort=False, ignore_index=True)
        
        sys.stdout.write('\r')
        sys.stdout.write('{}%' .format(round(100*(i+1)/data.shape[0],2)))
        sys.stdout.flush()
        
    return results


def add_pois(data, profile, nb_mins, isochrone_client=ISOCHRONE_CLIENT, pois_client=POIS_CLIENT):
    results = pd.DataFrame(dtype=np.int8)
    print(data.shape)
    for i, row in data.iterrows():
        location = [[row.longitude, row.latitude]]
        isochrome = get_isochrone(isochrone_client, location, profile, nb_mins)
        geometry = isochrome['features'][0]['geometry']
        list_pois = get_pois(pois_client, geometry, 'pois')['features']
        
        tmp = []
        for poi in list_pois:
            for cat, info in poi['properties']['category_ids'].items():
                tmp.append({'category':'{}/{}'.format(info['category_group'], info['category_name']),
                            'longitude':poi['geometry']['coordinates'][0],
                            'latitude':poi['geometry']['coordinates'][1]})
        pois = pd.DataFrame(tmp)
        pois.drop_duplicates(inplace=True)
        pois = pois.groupby(['category']).size()
        pois = pd.concat([row, pois])
        results = pd.concat([results, pois.to_frame().T], sort=False, ignore_index=True)
        
        sys.stdout.write('\r')
        sys.stdout.write('{}%' .format(round(100*(i+1)/data.shape[0],2)))
        sys.stdout.flush()
        
    return results

In [37]:
iso = get_isochrone(ISOCHRONE_CLIENT, TEST_LOCATION, ISOCHRONE_PROFILE, ISOCHRONE_NB_MINS)
pois = get_pois(POIS_CLIENT, iso['features'][0]['geometry'], 'pois')
pois

#[{'category':'{}/{}'.format(info['category_group'], info['category_name']), 'coordinates':poi['geometry']['coordinates']} for poi in pois['features'] for cat, info in poi['properties']['category_ids'].items()]

{'type': 'FeatureCollection',
 'bbox': [-1.603677, 47.208812, -1.603063, 47.209877],
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [-1.603582, 47.209877]},
   'properties': {'osm_id': 4309893301,
    'osm_type': 1,
    'distance': 0.0,
    'category_ids': {'583': {'category_name': 'bicycle_parking',
      'category_group': 'transport'}}}},
  {'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [-1.603626, 47.209847]},
   'properties': {'osm_id': 4323407097,
    'osm_type': 1,
    'distance': 0.0,
    'category_ids': {'167': {'category_name': 'fire_hydrant',
      'category_group': 'facilities'}}}},
  {'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [-1.603626, 47.209847]},
   'properties': {'osm_id': 4323407097,
    'osm_type': 1,
    'distance': 0.0,
    'category_ids': {'167': {'category_name': 'fire_hydrant',
      'category_group': 'facilities'}}}},
  {'type': 'Feature',
   'geometry': {'type': 'Point', 'coordina

In [83]:
df = add_pois(data.head(), ISOCHRONE_PROFILE, ISOCHRONE_NB_MINS)
df.to_pickle('store/with_poi.pkl.xz')

(5, 15)
100.0%